In [5]:

!jupyter nbconvert --to html --TemplateExporter.exclude_code_cell=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True scc2425-lab4.ipynb 2> /dev/null
!jupyter nbconvert --to slides --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True scc2425-lab4.ipynb 2> /dev/null

# Cloud Computing Systems
## 2024/25

Lab 4
https://smduarte.github.io/scc2425/

Sérgio Duarte, Kevin Gallagher 

# Goals

+ Create a Azure Redis for Cache account @ Azure;
+ Try AzureRedisCache to store various kinds of data in cache.
+ Continue converting Tukano to run on Azure; think of leveraging Redis to improve latency.

# Goals

+ **Create a Azure Redis for Cache account @ Azure;**
+ Try AzureRedisCache to store various kinds of data in cache.
+ Continue converting Tukano to run on Azure; think of leveraging Redis to improve latency.

# Create Azure Redis for Cache

<img src="redis-1.png" width="75%"></img>

# Create Azure Redis for Cache (2)

<img src="redis-2.png" width="75%"></img>

# Create Azure Redis for Cache (3)

<img src="redis-3.png" width="75%"></img>

# Create Azure Redis for Cache (4)

<img src="redis-4.png" width="75%"></img>

# Create Azure Redis for Cache (5)

<img src="redis-5.png" width="75%"></img>

# Create Azure Redis for Cache: URL

<img src="redis-6.png" width="75%"></img>

# Create Azure Redis for Cache: Key

<img src="redis-7.png" width="75%"></img>

# Goals

+ Create a Azure Redis for Cache account @ Azure;
+ **Try  AzureRedisCache to store various kinds of data in cache.**
+ Continue converting Tukano to run on Azure; think of leveraging Redis to improve latency.

# Accessing Azure Cache for Redis: useful links

We will use the library provided by Microsoft.

Java Docs available at:

+ [https://www.javadoc.io/doc/redis.clients/jedis/4.2.3](https://www.javadoc.io/doc/redis.clients/jedis/4.2.3)

Overview on how to use at:

+ [https://docs.microsoft.com/en-us/azure/azure-cache-for-redis/cache-java-get-started](https://docs.microsoft.com/en-us/azure/azure-cache-for-redis/cache-java-get-started)


# Redis datatypes

[https://redis.io/topics/data-types](https://redis.io/topics/data-types)

1. Key/Value String pairs
2. Lists and Sets (sorted and unsorted) of Strings
3. Counters
4. Hashes (similar to a struct or map)
5. Probabilistic data structures, such as HyperLogLogs, Bloom filters, Cuckoo filters, etc).

Most values need to be stored as strings. Easily resolved by using JSON as the external representation
of the data.

# Maven dependencies

This for for talking to Redis
```xml
<dependency>
	<groupId>redis.clients</groupId>
	<artifactId>jedis</artifactId>
	<version>5.0.1</version>
</dependency>
```

For serializing objects to JSON
```xml
<dependency>
	<groupId>com.fasterxml.jackson.core</groupId>
	<artifactId>jackson-core</artifactId>
	<version>2.15.2</version>
</dependency>
<dependency>
	<groupId>com.fasterxml.jackson.core</groupId>
	<artifactId>jackson-databind</artifactId>
	<version>2.15.2</version>
</dependency>
```

# Step 1: create client to Redis


### Alternatives:

1. Create a single client to Redis: 
    
    A client has a connection to the Redis server and it is not thread-safe.
    
    **- Not a good option for application servers.**<br><br>



2. Use a pool of clients to Redis, which can be shared by multiple threads.

# Step 1: create pool of clients to Redis

```java
private static JedisPool instance;

public synchronized static JedisPool getCachePool() {
    if( instance != null)
        return instance;
    
    var poolConfig = new JedisPoolConfig();
    poolConfig.setMaxTotal(128);
    poolConfig.setMaxIdle(128);
    poolConfig.setMinIdle(16);
    ...
    poolConfig.setBlockWhenExhausted(true);
    instance = new JedisPool(poolConfig, RedisHostname, REDIS_PORT, REDIS_TIMEOUT, RedisKey, REDIS_USE_TLS);
    return instance;
}
```

# Step 2: Use client from Jedis pool

Retrive a client from pool with a **[try with resources](https://docs.oracle.com/javase/8/docs/technotes/guides/language/try-with-resources.html)** block
```java
try (Jedis jedis = RedisCache.getCachePool().getResource()) {

	// Execute operations with a client to the Redis server

}
```
The client will be released automatically back to the pool...

# Step 3: Write and Read a key/value pair

```java
var user1 = new User("john", "1234", "john@nova.pt", "John Smith")

try (Jedis jedis = RedisCache.getCachePool().getResource()) {
    
    var key1 = "users:" + user.id();
    var value1 = JSON.encode( user1);
    jedis.set( key1, value2 );
	
    var key2 = ...
    var value2 = jedis.get( key2 );
    var user2 = JSON.decode( value2, User.class);
}

```

# Step 4: Operate on a List

```java
static MOST_RECENT_USERS_LIST = "MostRecentUsers";

var user1 = new User("john", "1234", "john@nova.pt", "John Smith")

try (Jedis jedis = RedisCache.getCachePool().getResource()) {
    
    var size = jedis.lpush(MOST_RECENT_USERS_LIST, JSON.encode(user1) );
    if (size > 5)
        jedis.ltrim(MOST_RECENT_USERS_LIST, 0, 4);

    var list = jedis.lrange(MOST_RECENT_USERS_LIST, 0, -1);
    
    System.out.println(MOST_RECENT_USERS_LIST);
    for( String s : list)
        System.out.println(s);
}

```

Note: Redis Lists support several operations, such as appending, trimming and retrieving range...

# Step 4: Increment a Counter

```java
static final String MUM_USERS_COUNTER = "NumUsers";

try (Jedis jedis = RedisCache.getCachePool().getResource()) {
    var cnt = jedis.incr(MUM_USERS_COUNTER);
    System.out.println( "Num users : " + cnt);
}

```

Note: Redis counters are updated atomically and support concurrent increments by multiple clients...

# Step 4: Redis key expiration


+ **expire( key, seconds)**

Allows to set an expiration time for a key. 

After the time for the key expires, the key is automatically deleted from the cache.

Resetting the timeout is not automatic for all operations. 

For example, incrementing a key does not clear the timeout and postpone expiration.

Check the documentation.

# Sample Code

The code provided [scc2425-lab4.zip](scc2425-lab4.zip) is a Maven project for uploading and downloading file to and from the Blob store (directly).

For testing it in the command line, just run:

```mvn clean compile assembly:single```

This will compile and create a single file with all compiled classes and dependencies.

Run the program as follows:

```java -cp target/scc2425-lab4-1.0-jar-with-dependencies.jar scc.utils.TestCache```

# Goals

+ Create a Azure Redis for Cache account @ Azure;
+ Try AzureRedisCache to store various kinds of data in cache.
+ **Continue converting Tukano to run on Azure; think of leveraging Redis to improve latency.**